# 🧠 CNN: Convolutional Neural Networks - Sarrera

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Maila**: Aurreratua  

---

## 🎯 Helburua

Notebook honetan ikasiko dugu:
- Zer dira CNN sare neuronalak
- Nola funtzionatzen duten konboluzio geruza
- CNN vs MLP (Perceptron Multikapa) konparaketa
- Lehen CNN eredua MNIST dataset-arekin

---

## 📚 Gako-Kontzeptuak

### CNN vs Sare Neuronal Tradizionala

| Ezaugarria | MLP (Tradizionala) | CNN |
|------------|-------------------|-----|
| Sarrera | Bektore (1D) | Irudi (2D/3D) |
| Konexioak | Osoak | Lokalak |
| Parametroak | Asko | Gutxiago |
| Irudietarako | ❌ Ez oso egokia | ✅ Optimoa |

### CNN Geruza Motak

1. **Convolutional Layer**: Karakteristikak atera (ertzak, texturak)
2. **Pooling Layer**: Dimentsioa murriztu
3. **Activation Layer**: Non-linealtasuna gehitu (ReLU)
4. **Fully Connected Layer**: Klasifikazioa

## 1️⃣ Liburutegiak Inportatu

In [ ]:
# Liburutegi nagusiak
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical

# Konfigurazioa
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Liburutegiak kargatuta!")
print(f"📦 TensorFlow bertsioa: {keras.__version__}")

## 2️⃣ MNIST Dataset-a Kargatu

MNIST dataset-ak 70.000 eskuz idatzitako zenbakien irudi ditu:
- 60.000 entrenamendu irudiak
- 10.000 test irudiak
- Tamaina: 28x28 pixel (eskala grisean)

In [ ]:
# Dataset-a kargatu
print("📥 MNIST dataset-a deskarga hasita...")
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"\n📊 Dataset Informazioa:")
print(f"  Entrenamendu irudiak: {X_train.shape[0]:,}")
print(f"  Test irudiak: {X_test.shape[0]:,}")
print(f"  Irudi tamaina: {X_train.shape[1]}x{X_train.shape[2]} pixel")
print(f"  Klase kopurua: {len(np.unique(y_train))} (0-9 zenbakiak)")

## 3️⃣ Datuak Bistaratu

In [ ]:
# 25 irudi bistaratu
fig, axes = plt.subplots(5, 5, figsize=(12, 12))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i], cmap='gray')
    ax.set_title(f"Zenbakia: {y_train[i]}", fontsize=12, color='navy')
    ax.axis('off')
    
plt.suptitle('🔢 MNIST Dataset - Adibide Irudiak', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("✅ Irudiak bistaratuta!")

## 4️⃣ MLP vs CNN Konparaketa

Ikusiko dugu nola **CNN** hobea den **MLP** (Multi-Layer Perceptron) baino irudi-sailkapenerako.

In [ ]:
# Datuak prestatu
X_train_norm = X_train / 255.0
X_test_norm = X_test / 255.0

# MLP Eredua (oinarrizkoa)
print("🔷 MLP Eredua eraikitzen...")
mlp_model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
], name='MLP')

# CNN Eredua (aurreratua)
print("🔶 CNN Eredua eraikitzen...")
cnn_model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
], name='CNN')

# Konpilatu
mlp_model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

cnn_model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

print("\n✅ Bi ereduak prestatutak!")

In [ ]:
# MLP entrenatu
print("🔹 MLP entrenamendu hasita...")
mlp_history = mlp_model.fit(
    X_train_norm, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

# CNN entrenatu
print("\n🔹 CNN entrenamendu hasita...")
cnn_history = cnn_model.fit(
    X_train_norm.reshape(-1, 28, 28, 1), y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

## 5️⃣ Emaitzak Konparatu

In [ ]:
# Test accuracy kalkulatu
mlp_test_loss, mlp_test_acc = mlp_model.evaluate(X_test_norm, y_test, verbose=0)
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(X_test_norm.reshape(-1, 28, 28, 1), y_test, verbose=0)

# Emaitzak
print("\n" + "="*50)
print("📊 EMAITZA FINALA")
print("="*50)
print(f"\n🔷 MLP Model:")
print(f"   Validation Accuracy: {mlp_history.history['val_accuracy'][-1]:.4f}")
print(f"   Test Accuracy: {mlp_test_acc:.4f}")
print(f"\n🔶 CNN Model:")
print(f"   Validation Accuracy: {cnn_history.history['val_accuracy'][-1]:.4f}")
print(f"   Test Accuracy: {cnn_test_acc:.4f}")
print(f"\n✅ CNN HOBEA DA: {(cnn_test_acc - mlp_test_acc)*100:.2f}% hobetze!")
print("="*50)

In [ ]:
# Grafiko konparatiboa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(mlp_history.history['accuracy'], label='MLP Train', linewidth=2)
axes[0].plot(mlp_history.history['val_accuracy'], label='MLP Validation', linestyle='--', linewidth=2)
axes[0].plot(cnn_history.history['accuracy'], label='CNN Train', linewidth=2)
axes[0].plot(cnn_history.history['val_accuracy'], label='CNN Validation', linestyle='--', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('📈 Accuracy Konparaketa', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(mlp_history.history['loss'], label='MLP Train', linewidth=2)
axes[1].plot(mlp_history.history['val_loss'], label='MLP Validation', linestyle='--', linewidth=2)
axes[1].plot(cnn_history.history['loss'], label='CNN Train', linewidth=2)
axes[1].plot(cnn_history.history['val_loss'], label='CNN Validation', linestyle='--', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('📉 Loss Konparaketa', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6️⃣ Ondorioak

### 🎯 Ikasitakoa:

1. **CNN hobeak dira irudietarako**: Espazialki lotutako patroiak detektatzeko gai dira
2. **Parametro gutxiago**: CNN-k parametro gutxiago ditu MLP-k baino
3. **Generalizazio hobea**: CNN-k overfitting-a gehiago saihesten dute

### 📚 Hurrengo Pausua:

Notebook honetan CNN-en oinarriak ikasi ditugu. Hurrengo notebook-ean:
- 🔄 **Transfer Learning** teknikak
- 🎯 **Pre-trained ereduak** (ResNet, VGG16)
- 🚀 **Fine-tuning** teknikak

---

**Zorionak! ✨ Lehen CNN notebook-a osatu duzu!**